# MCCE program template
*By Junjun Mao <junjun.mao@gmail.com>*

Feel free to use this template to run pKa calculation with MCCE.

A complete documentation is at https://gunnerlab.github.io/Stable-MCCE/

## Prepare the running environment

MCCE program needs to know where mcce executables are. This part set the environment.

<span style="color:red">Change MCCE_PATH to your installation folder unless you run this on Levich server.</span>

In [ ]:
# Point this to your MCCE installation directory 
MCCE_PATH = "/home/mcce/Stable-MCCE"

In [ ]:
import sys
import os
import subprocess

# Add to environment PATH
os.environ["PATH"] = "%s/bin/:%s" % (MCCE_PATH, os.environ["PATH"])
# Add to Python PATH
sys.path.insert(0, "%s/bin" % MCCE_PATH)

## Get the pdb file

You can download from PDB like in this example, or upload a file to this directory.

In [ ]:
!getpdb 4lzt

## Step 1. Convert PDB file into MCCE PDB

This step proof reads the structure file and cut terminal residues and complex cofactors into smaller ones if necessary.

In [ ]:
!step1.py 4LZT.pdb

## Step 2. Make side chain conformers

This step makes alternative side chain locations and ionization states.

In [ ]:
!step2.py

## Step 3. Make energy table

This step calculates conformer self energy and pairwise interaction table.

You can run step 3 assuming the protein has dielectric constant 4 or 8. Small proteins have mor ploar and ionizable residues so  dielectric constant 8 is often better.

**This step may take hours, please be patient.**

In [ ]:
!step3.py

# Use this line for dielectric constant 8
#!step3.py -d 8

# Step 4. Simulate a titration with Monte Carlo sampling

This step simulates a titration and writes out the conformation and ionization states of each side chain at various conditions.

Use command
```
step4.py --xts
```
if you want entropy correction, which makes ionized and neutral groups of conformers have an equal chance to be sampled.

Or
```
step4.py
```
without entropy correction.

In [ ]:
!step4.py --xts
#!step4.py

## Check the results 

* The occupancy table is in file fort.38.
* The net charge is in file sum_crg.out
* pKas are in file pK.out

In [ ]:
#check calculated pKa values
!cat pK.out 

In [ ]:
#Checkcomformer occupancy
!cat fort.38

In [ ]:
#checkresidue charges
!cat sum_crg.out

## Fit the titration curve
Use the residue IDs in sum_crg.out or pK.out, pass to fitpka function, you can get a fitted titration curves.

In [ ]:
import fitpka
fitpka.fitpka(["GLU-A0035_", "ASP-A0066_"])

## Show major influencers to a residue pKa

Use mfe.py to see all terms that contribute to the ionization free energy. https://gunnerlab.github.io/Stable-MCCE/tools/#mfepy

In [ ]:
!mfe.py GLU-A0035_ -c 0.1